In [1]:
import gurobipy as gb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import copy
import random

plt.rcParams['font.size']=12
plt.rcParams['font.family']='serif'
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False  
plt.rcParams['axes.spines.bottom'] = True     
plt.rcParams["axes.grid"] = True
plt.rcParams['grid.linestyle'] = '-.' 
plt.rcParams['grid.linewidth'] = 0.4

Set up constants

In [2]:
OMEGA = 250 # number of scenarios to sample
PI = 1 / OMEGA # probability of each sampled scenario - assumed to be equal


T = 24 # number of hours

WIND_CAPACITY = 200 #MWh

#CVaR parameters
BETAS = np.arange(0, 1.1, 0.1)
ALPHA = 0.9

# Read scenario data

Refer to **read_data.ipynb** for insight regarding how the data is generated and structured.

In [3]:
with open('Data/ALL_scenarios.json') as f:
    all_scenarios = json.load(f)

S = len(all_scenarios.keys()) - 1 # number of total scenarios

random.seed(123)

# Sample scenarios without replacement
in_sample_scenarios = random.sample(range(S), OMEGA)

print(in_sample_scenarios)

scenarios = {}

# Extract sampled scenarios from dictionary containing all scenarios
for i in range(len(in_sample_scenarios)):
    scenarios[str(i)] = all_scenarios[str(in_sample_scenarios[i])]
    scenarios[str(i)]['Original Index'] = in_sample_scenarios[i]
    
print('Number of extracted scenarios:', len(scenarios))

[107, 548, 178, 1574, 834, 545, 220, 1717, 1845, 1791, 78, 776, 1098, 1151, 680, 697, 1745, 106, 326, 276, 690, 1148, 683, 1437, 502, 335, 3, 1855, 893, 1584, 179, 1799, 1223, 773, 143, 13, 646, 1499, 918, 208, 1854, 89, 189, 1366, 291, 258, 1618, 1856, 43, 597, 1866, 881, 1174, 976, 543, 961, 1715, 75, 624, 703, 1068, 1641, 989, 423, 1681, 1247, 1308, 1079, 1156, 1657, 1967, 645, 25, 815, 1828, 1575, 1323, 1050, 889, 1402, 1102, 1307, 1631, 1365, 1221, 1822, 1707, 995, 1066, 1369, 859, 767, 1044, 65, 1735, 1544, 1321, 371, 1486, 1679, 169, 1871, 997, 1347, 536, 348, 685, 1135, 801, 1810, 150, 1480, 935, 793, 1173, 1594, 688, 33, 387, 1200, 180, 1703, 1286, 744, 26, 1835, 720, 469, 830, 1089, 1026, 1482, 1687, 1861, 122, 589, 1013, 1974, 1565, 1290, 1475, 477, 1114, 990, 363, 314, 910, 775, 1539, 944, 90, 20, 275, 934, 395, 525, 1636, 556, 1955, 1298, 860, 0, 222, 1264, 128, 1045, 727, 1448, 181, 887, 1838, 17, 1629, 1097, 1119, 1421, 1300, 827, 59, 660, 537, 1957, 967, 67, 1537, 1150,

# One-price Scheme (untouched for now)

In [7]:
def solve_op_scheme(scenarios, WIND_CAPACITY, T, OMEGA):
    direction = gb.GRB.MAXIMIZE #Min / Max

    m = gb.Model() # Create a Gurobi model  
    m.setParam('OutputFlag', 0)

    #============= Variables =============
    p_DA = m.addVars(T, lb=0, ub=gb.GRB.INFINITY, name="p_DA") # day-ahead power bid
    delta = m.addVars(T, OMEGA, lb=-gb.GRB.INFINITY, ub=gb.GRB.INFINITY, name="delta") # decision variable for the power imbalance - can be negative
    price_coeff = m.addVars(T, OMEGA, lb=0, ub=gb.GRB.INFINITY, name="K") # price coefficient for the imbalance price wrt. the day-ahead price

    #============= Objective function =============
    # Set objective function - note that the day-ahead price is factored out of the sum
    obj = gb.quicksum(PI * scenarios[str(w)]['Spot Price [EUR/MWh]'][t] * (p_DA[t] + price_coeff[t,w] * delta[t,w]) for t in range(T) for w in range(OMEGA))
    m.setObjective(obj, direction)

    #============= Day-ahead power bid limits ============

    #Upper limit is the nominal wind power
    m.addConstrs(p_DA[t] <= WIND_CAPACITY for t in range(T))

    #============= Power imbalance definition (realized - bid) ===============
    m.addConstrs(delta[t,w] == scenarios[str(w)]['Wind Power [MW]'][t] - p_DA[t] for t in range(T) for w in range(OMEGA))

    #============= Price coefficient definition ===============
    # the system balance parameter is 0 if the system has a surplus and 1 if it has a deficit
    m.addConstrs(price_coeff[t,w] == 1.2 * scenarios[str(w)]['System Balance State'][t] + 0.9 * (1 - scenarios[str(w)]['System Balance State'][t]) for t in range(T) for w in range(OMEGA))

    #============= Display and run model =============
    m.update()
    #m.display()
    m.optimize()

    #============= Results =============
    results = {}
    if m.status == gb.GRB.OPTIMAL:
        #initialization
        for scenario in range(OMEGA):
            df = pd.DataFrame(columns=['Hour', 'DA Price [EUR/MWh]', 'Wind Power [MW]', 'DA Bid [MW]', 'Imbalance [MW]', 'DA Profit [EUR]', 'Balancing Profit [EUR]', 'System State', 'Balancing Price Coefficient'])
            
            for t in range(T):
                df.loc[t] = [t, 
                            scenarios[str(scenario)]['Spot Price [EUR/MWh]'][t], 
                            scenarios[str(scenario)]['Wind Power [MW]'][t], p_DA[t].x, 
                            delta[t,scenario].x, scenarios[str(scenario)]['Spot Price [EUR/MWh]'][t] * p_DA[t].x, 
                            price_coeff[t,scenario].x * scenarios[str(scenario)]['Spot Price [EUR/MWh]'][t] * delta[t,scenario].x, 
                            scenarios[str(scenario)]['System Balance State'][t], price_coeff[t,scenario].x]
            df['Total Profit [EUR]'] = df['DA Profit [EUR]'] + df['Balancing Profit [EUR]']

            df['Hour'] = df['Hour'].astype(int)
            df['System State'] = df['System State'].astype(int)
            df['System State'] = df['System State'].apply(lambda x: 'Deficit' if x == 1 else 'Surplus')
            df.set_index('Hour', inplace=True)
            results[scenario] = df.copy(deep=True)

        print('-----------------------------------------------')
        print('Objective value (expected profit): %.2f EUR' % m.objVal)
        print('-----------------------------------------------')
        print('Day-ahead bids:')
        average_hourly_profit = np.mean([results[w]['Total Profit [EUR]'] for w in range(OMEGA)], axis=0)

        summary = pd.DataFrame(columns=['Hour', 'DA Bid [MW]', 'Average Profit [EUR]', 'Average Wind [MW]', 'Median Wind [MW]', 'Average System State', 'Average Price Coefficient'])
        for t in range(T):
            summary.loc[t] = [t, p_DA[t].x, np.mean(average_hourly_profit[t]), np.mean([results[w]['Wind Power [MW]'][t] for w in range(OMEGA)]), np.median([results[w]['Wind Power [MW]'][t] for w in range(OMEGA)]), np.mean([scenarios[str(w)]['System Balance State'][t] for w in range(OMEGA)]), np.mean([price_coeff[t,w].x for w in range(OMEGA)])]

        summary['Hour'] = summary['Hour'].astype(int)
        summary.set_index('Hour', inplace=True)

        results['Summary'] = summary.copy(deep=True)

        for t in range(T):
            print('Hour %d | Bid: %.2f MW | Average Profit: %.2f EUR' % (t, p_DA[t].x, average_hourly_profit[t]))

        print('Sum of average profits: %.2f EUR' % np.sum(average_hourly_profit))
        print('-----------------------------------------------')
        print('Runtime: %f ms' % (m.Runtime * 1e3))
        return p_DA, results
    else:
        print("Optimization was not successful.")
        return None, None    


In [ ]:
p_DA_op, results_op = solve_op_scheme(scenarios, WIND_CAPACITY, T, OMEGA)

# Two-price Scheme

In [4]:
def solve_tp_scheme_cvar(scenarios, beta_values, alpha, WIND_CAPACITY, T, OMEGA, minimize_printouts=False, mip_gap = 1e-4):

    results_per_beta = {}

    for beta in beta_values:
        
        print('===============================================')
        print('Solving for beta = %.2f...' % beta)

        direction = gb.GRB.MAXIMIZE #Min / Max

        m = gb.Model() # Create a Gurobi model  

        m.setParam('OutputFlag', 0)
        m.setParam('MIPGap', mip_gap)

        #============= Variables =============
        p_DA = m.addVars(T, lb=0, ub=gb.GRB.INFINITY, name="p_DA") # day-ahead power bid

        delta = m.addVars(T, OMEGA, lb=-gb.GRB.INFINITY, ub=gb.GRB.INFINITY, name="delta") # decision variable for the power imbalance - can be negative
        delta_up = m.addVars(T, OMEGA, lb=0, ub=gb.GRB.INFINITY, name="delta_up") # surplus
        delta_down = m.addVars(T, OMEGA, lb=0, ub=gb.GRB.INFINITY, name="delta_down") # deficit
        var = m.addVar(lb=-gb.GRB.INFINITY, ub=gb.GRB.INFINITY, name="VaR") # value at risk
        eta = m.addVars(OMEGA, lb=0, ub=gb.GRB.INFINITY, name="eta") # auxiliary variable for CVaR

        imbalance_revenue = m.addVars(T, OMEGA, lb=-gb.GRB.INFINITY, ub=gb.GRB.INFINITY, name="I") # imbalance revenue - can be negative

        # binary variables used to control the two-price logic
        y = m.addVars(T, OMEGA, vtype=gb.GRB.BINARY, name="y")
        z = m.addVars(4, T, OMEGA, vtype=gb.GRB.BINARY, name="z")

        #============= Objective function =============
        # Set objective function
        expected_profit = gb.quicksum(PI * (scenarios[str(w)]['Spot Price [EUR/MWh]'][t] * p_DA[t] + imbalance_revenue[t,w]) for t in range(T) for w in range(OMEGA))
        cvar = (var - ((1 / (1 - alpha)) * gb.quicksum(PI * eta[w] for w in range(OMEGA))))

        obj = (1 - beta) * expected_profit + beta * cvar

        m.setObjective(obj, direction)

        #============= Day-ahead power bid limits ============

        #Upper limit is the nominal wind power
        m.addConstrs(p_DA[t] <= WIND_CAPACITY for t in range(T))

        #============= Power imbalance definitions ===============
        m.addConstrs(delta[t,w] == scenarios[str(w)]['Wind Power [MW]'][t] - p_DA[t] for t in range(T) for w in range(OMEGA))
        m.addConstrs(delta[t,w] == delta_up[t,w] - delta_down[t,w] for t in range(T) for w in range(OMEGA))


        M = 1e6 # big-M constant
        #ensure that only one of the delta directions can be non-zero
        m.addConstrs(delta_up[t,w] <= M * (1 - y[t,w]) for t in range(T) for w in range(OMEGA))
        m.addConstrs(delta_down[t,w] <= M * y[t,w] for t in range(T) for w in range(OMEGA))

        #============= Linearized conditional statements ===============
        #Binary variable constraints
        m.addConstrs(z[0,t,w] <= y[t,w] + scenarios[str(w)]['System Balance State'][t] for t in range(T) for w in range(OMEGA))
        m.addConstrs(z[1,t,w] <= y[t,w] + (1 - scenarios[str(w)]['System Balance State'][t]) for t in range(T) for w in range(OMEGA))
        m.addConstrs(z[2,t,w] <= (1 - y[t,w]) + scenarios[str(w)]['System Balance State'][t] for t in range(T) for w in range(OMEGA))
        m.addConstrs(z[3,t,w] <= (1 - y[t,w]) + (1 - scenarios[str(w)]['System Balance State'][t]) for t in range(T) for w in range(OMEGA))

        # if system is in a surplus and the imbalance is positive (NOT helping the system)
        m.addConstrs(imbalance_revenue[t,w] <= 0.9 * scenarios[str(w)]['Spot Price [EUR/MWh]'][t] * delta_up[t,w] + M * z[0,t,w] for t in range(T) for w in range(OMEGA))

        # if system is in a deficit and the imbalance is positive (IS helping the system)
        m.addConstrs(imbalance_revenue[t,w] <= 1.0 * scenarios[str(w)]['Spot Price [EUR/MWh]'][t] * delta_up[t,w] + M * z[1,t,w] for t in range(T) for w in range(OMEGA))

        # if system is in a surplus and the imbalance is negative (IS helping the system)
        m.addConstrs(imbalance_revenue[t,w] <= -1.0 * scenarios[str(w)]['Spot Price [EUR/MWh]'][t] * delta_down[t,w] + M * z[2,t,w] for t in range(T) for w in range(OMEGA))

        # if system is in a deficit and the imbalance is negative (NOT helping the system)
        m.addConstrs(imbalance_revenue[t,w] <= -1.2 * scenarios[str(w)]['Spot Price [EUR/MWh]'][t] * delta_down[t,w] + M * z[3,t,w] for t in range(T) for w in range(OMEGA))

        #============= Conditional value at risk (CVaR) definition ===============
        # CVaR is the expected value of the tail of the distribution
        m.addConstrs(-1 * gb.quicksum(PI * (scenarios[str(w)]['Spot Price [EUR/MWh]'][t] * p_DA[t] + imbalance_revenue[t,w]) for t in range(T)) + var - eta[w] <= 0 for w in range(OMEGA))

        #============= Display and run model =============
        m.update()
        #m.display()
        m.optimize()

        #============= Results =============
        results = {}
        if m.status == gb.GRB.OPTIMAL:
            #initialization
            for scenario in range(OMEGA):
                df = pd.DataFrame(columns=['Hour', 'DA Price [EUR/MWh]', 'Wind Power [MW]', 'DA Bid [MW]', 'Delta [MW]', 'Delta UP [MW]', 'Delta DOWN [MW]' ,'DA Profit [EUR]', 'Balancing Profit [EUR]', 'System State'])
                
                for t in range(T):
                    df.loc[t] = [t, 
                                scenarios[str(scenario)]['Spot Price [EUR/MWh]'][t], 
                                scenarios[str(scenario)]['Wind Power [MW]'][t], p_DA[t].x,
                                delta[t,scenario].x, 
                                delta_up[t,scenario].x, 
                                delta_down[t,scenario].x, 
                                scenarios[str(scenario)]['Spot Price [EUR/MWh]'][t] * p_DA[t].x, 
                                imbalance_revenue[t,scenario].x, 
                                scenarios[str(scenario)]['System Balance State'][t]]
                df['Total Profit [EUR]'] = df['DA Profit [EUR]'] + df['Balancing Profit [EUR]']

                df['Hour'] = df['Hour'].astype(int)
                df['System State'] = df['System State'].astype(int)
                df['System State'] = df['System State'].apply(lambda x: 'Deficit' if x == 1 else 'Surplus')
                df.set_index('Hour', inplace=True)
                results[scenario] = df.copy(deep=True)

            print('-----------------------------------------------')
            print('Objective value: %.2f EUR' % m.objVal)
            print('Expected profit: %.2f EUR' % (expected_profit.getValue()))
            print('CVaR: %.2f EUR' % (cvar.getValue()))
            print('VaR: %.2f EUR' % (var.x))
            
            if not minimize_printouts:
                print('-----------------------------------------------')
                print('Day-ahead bids:')
                average_hourly_profit = np.mean([results[w]['Total Profit [EUR]'] for w in range(OMEGA)], axis=0)

                summary = pd.DataFrame(columns=['Hour', 'DA Bid [MW]', 'Average Profit [EUR]', 'Average Wind [MW]', 'Median Wind [MW]', 'Average System State'])
                for t in range(T):
                    summary.loc[t] = [t, p_DA[t].x, np.mean(average_hourly_profit[t]), np.mean([results[w]['Wind Power [MW]'][t] for w in range(OMEGA)]), np.median([results[w]['Wind Power [MW]'][t] for w in range(OMEGA)]), np.mean([scenarios[str(w)]['System Balance State'][t] for w in range(OMEGA)])]

                summary['Hour'] = summary['Hour'].astype(int)
                summary.set_index('Hour', inplace=True)

                results['Summary'] = summary.copy(deep=True)

                for t in range(T):
                    print('Hour %d | Bid: %.2f MW | Average Profit: %.2f EUR' % (t, p_DA[t].x, average_hourly_profit[t]))

                print('Sum of average profits: %.2f EUR' % np.sum(average_hourly_profit))
                print('-----------------------------------------------')
                print('Runtime: %f ms' % (m.Runtime * 1e3))

            for scenario in range(OMEGA):
                for t in range(T):
                    if np.round(sum([z[i,t,scenario].x for i in range(4)]), 4) != 3:
                        print('WARNING: SCENARIO %d | HOUR %d | z:' % (scenario, t), z[0,t,scenario].x, z[1,t,scenario].x, z[2,t,scenario].x, z[3,t,scenario].x)
            
            results['Expected Profit'] = expected_profit.getValue()
            results['CVaR'] = cvar.getValue()
            results['VaR'] = var.x

            m.dispose() #dispose of current model       
        else:
            print("Optimization was not successful.") 
            return None

        results_per_beta[beta] = copy.deepcopy(results)

    return results_per_beta

In [5]:
results_tp = solve_tp_scheme_cvar(scenarios, BETAS, ALPHA, WIND_CAPACITY, T, OMEGA, minimize_printouts=True, mip_gap=0.01)

Solving for beta = 0.00...
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-28
-----------------------------------------------
Objective value: 151998.56 EUR
Expected profit: 151998.56 EUR
CVaR: 22.00 EUR
VaR: 22.00 EUR
Solving for beta = 0.10...
-----------------------------------------------
Objective value: 136804.62 EUR
Expected profit: 151998.56 EUR
CVaR: 59.10 EUR
VaR: 76.68 EUR
Solving for beta = 0.20...
-----------------------------------------------
Objective value: 121610.67 EUR
Expected profit: 151998.56 EUR
CVaR: 59.10 EUR
VaR: 76.68 EUR
Solving for beta = 0.30...
-----------------------------------------------
Objective value: 106416.72 EUR
Expected profit: 151998.56 EUR
CVaR: 59.10 EUR
VaR: 76.68 EUR
Solving for beta = 0.40...
-----------------------------------------------
Objective value: 91222.78 EUR
Expected profit: 151998.56 EUR
CVaR: 59.10 EUR
VaR: 76.68 EUR
Solving for beta = 0.50...
-------------------------------------------

# Export optimal day-ahead schedule for subsequent tasks

In [26]:
#Create dataframe with optimal day-ahead bids for each approach
df = pd.DataFrame(columns=['Hour', 'One-price Bids [MW]', 'Two-price Bids [MW]'])
df['Hour'] = range(T)
df['One-price Bids [MW]'] = results_op['Summary']['DA Bid [MW]'].values
df['Two-price Bids [MW]'] = results_tp['Summary']['DA Bid [MW]'].values
df.set_index('Hour', inplace=True)

#df.to_csv('Data/Optimal_DA_bids.csv')

df

,One-price Bids [MW],Two-price Bids [MW]
Hour,,
0,0.0,34.429065
1,0.0,17.363054
2,0.0,25.516105
3,0.0,27.733224
4,0.0,52.012413
5,0.0,48.027292
6,200.0,110.630951
7,0.0,44.485352
8,200.0,82.469291
